In [ ]:
from azmeta.access import resource_graph, monitor_logs, list_subscription_ids
from azmeta.notebook.context import AzureCliContext
import azmeta.notebook.interactive as azmi
import pandas as pd
import itertools

# Parameters

In [ ]:
resource_filter = None

# Log Analytics Workspace Selection

In [ ]:
context = AzureCliContext()
all_subscription_ids = list_subscription_ids(context.subscriptions)
workspaces = resource_graph.query_dataframe(all_subscription_ids, f"""
Resources 
| where type == 'microsoft.operationalinsights/workspaces'
| where {resource_filter if resource_filter else "1 == 1"}
| join (ResourceContainers | where type == 'microsoft.resources/subscriptions' | project subscriptionName=name, subscriptionId) on subscriptionId
| project subscriptionName, resourceGroup, name, sku = properties.sku.name, reservedGB = properties.sku.capacityReservationLevel, storeDays = properties.retentionInDays, id = properties.customerId, resourceId = tolower(id)
| order by subscriptionName asc
""").set_index('id')

In [ ]:
workspaces.style.hide_columns('resourceId')

# Pull Workspace Utilization

In [ ]:
def la_query(query):
    return monitor_logs.query_dataframe(query, workspaces.index.to_list()).primary_result.set_index('id')

df_1d = la_query("""
Usage
| where TimeGenerated > startofday(ago(1d)) and TimeGenerated <= startofday(now())
| where IsBillable == true
| summarize lastFullDayGB = sum(Quantity) / 1000 by TenantId
| project-rename id = TenantId
""")

In [ ]:
df_30d = la_query("""
Usage
| where TimeGenerated > startofday(ago(30d)) and TimeGenerated <= startofday(now())
| where IsBillable == true
| summarize fullDayGB = sum(Quantity) / 1000 by TenantId, bin(TimeGenerated, 1d)
| summarize medianDayGB = percentile(fullDayGB, 50) by TenantId 
| project-rename id = TenantId
""")

In [ ]:
df_nodes = la_query("""
Heartbeat
| where TimeGenerated > startofday(ago(1d)) and TimeGenerated <= startofday(now())
| summarize by SourceComputerId, TenantId
| summarize nodesReporting = count() by TenantId
| project-rename id = TenantId 
""")

# Pull Cost Data

In [ ]:
azmi.connect_kqlmagic()
workspace_resource_ids = workspaces.resourceId.to_list()

In [ ]:
%%kql cost_data << -ecbp
let period = toscalar(Usage
| summarize max(BillingPeriodStartDate));
Usage
| where BillingPeriodStartDate == period and ResourceId in~ ({workspace_resource_ids}) 
| summarize Cost=sum(Cost) by ResourceId, Date 
| summarize medianDay=percentile(Cost, 50), billPeriod=sum(Cost) by ResourceId
| project-rename resourceId=ResourceId;
print period=period

In [ ]:
df_cd = cost_data.to_dataframe()

# Report

In [ ]:
full = workspaces \
    .join([df_1d, df_30d, df_nodes]) \
    .merge(df_cd, on='resourceId')
full = full.assign(periodAvgCostPerNode=full.billPeriod/full.nodesReporting) 

In [ ]:
print("Billing Period: ", cost_data.fork_result(1).to_dataframe().iat[0, 0].strftime("%B %Y"))

In [ ]:
full.style.hide_index().hide_columns('resourceId').format('${:,.2f}', na_rep='N/A', subset=pd.IndexSlice[:,'medianDay':'periodAvgCostPerNode'])